# gradient
在pytorch的计算流图中，只有叶子节点的grad才会在backward()后有值，其余节点的grad都为None

In [26]:
import torch 
# x is leaf
x = torch.tensor([1.,2.,3.], requires_grad=True)
print(x.grad)
# y not leaf
y = x.sigmoid()
print(y)
# w is leaf
w = torch.randn(1,3, requires_grad=True)
z = y * w
# output and backward
o = z.sum()
o.backward()
# print results
print(x.grad, x.is_leaf)
print(y.grad,y.is_leaf)
print(w.grad,w.is_leaf)

None
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward>)
tensor([ 0.2935,  0.0508, -0.0225]) True
None False
tensor([[0.7311, 0.8808, 0.9526]]) True


## Variable
Variable是为了构架计算流图诞生的，其有三个属性：
- data: Tensor本身
- grad: Tensor的梯度
- grad_fn：对应一个Function

```
from torch.autograd import Variable
```

### Variable与Tensor的区别
- Tensor对象支持在原对象内存区域上修改数据
- Variable不支持在原对象内存区域上修改数据
- 对Variable对象的操作，操作会被记录,可通过grad_fn属性查看上一次的操作，可通过data属性访问原始张量


## requires_grad
tensor的requires_grad参数，如果为True，那么在反向传播时，会自动求导，

## leaf
- 当grad_fn为None时，无论requires_grad为True还是False，都为叶子变量，即只要是直接初始化的就为叶子变量
- 当grad_fn不为None时，requires_grad = False为叶子变量，requires_grad = True为非叶子变量

情况1: 直接初始化的都为叶子变量

grad_fn=None

In [4]:
x = torch.tensor([1,2,3], dtype=torch.float, requires_grad=True)
x.is_leaf

True

情况2：通过运算，生成新的tensor

grad_fn ！=None，requires_grad = True

In [9]:
y = x**2
y.is_leaf, y.requires_grad, y.grad_fn

(False, True, <PowBackward0 at 0x2b7999867f0>)

情况3：基于运算操作生成的tensor，只有当任意一个叶子节点requires_grad!=False时，才能生成非叶子节点

In [7]:
w = torch.randn(2, 3, requires_grad = False)
z = torch.randn(2, 3, requires_grad = False)
u = w + z
u.is_leaf, u.requires_grad

(True, False)

In [8]:
w = torch.randn(2, 3, requires_grad = False)
z = torch.randn(2, 3, requires_grad = True)
u = w + z
u.is_leaf, u.requires_grad

(False, True)

## detach
当我们在训练网络的时候可能希望保持一部分的网络参数不变，只对其中一部分的参数进行调整；或者只训练部分分支网络，并不让其梯度对主网络的梯度造成影响，这时候我们就需要使用detach()函数来切断一些分支的反向传播

作用：
- 返回一个从当前计算图中分离下的Variable，但与源Variable指向同一个tensor，返回的Variable永远不会需要梯度
- 将requires_grad置为False，将grad_fn置为None

In [35]:
import torch 
x = torch.tensor([1.,2.,3.], requires_grad=True)
print(x.grad)
y = x.sigmoid()
print(y)
z = y.sum()
print(f'z:{z},{z.requires_grad},{z.grad_fn}')
z_d = z.detach()
print(f'z_d:{z_d},{z_d.requires_grad},{z_d.grad_fn}')
z.backward()


None
tensor([0.7311, 0.8808, 0.9526], grad_fn=<SigmoidBackward>)
z:2.564429759979248,True,<SumBackward0 object at 0x000002B799986FD0>
z_d:2.564429759979248,False,None


1. 不能对detach的Variable进行backward

In [38]:
z_d.backward()

RuntimeError: element 0 of tensors does not require grad and does not have a grad_fn

2. 对detach出的Variable进行修改，也会修改原Variable对应的tensor

In [37]:
z_d.zero_()
print(z_d.data)
print(z.data)

tensor(0.)
tensor(0.)


### demo

In [49]:
import torch
import torch.nn as nn
layer1 = nn.Conv2d(3,1,(3,3))
layer2 = nn.Conv2d(1,1,(3,3))
'''
#第一种方法
'''
# 网络层1
x = torch.randn(1,3,32,32, requires_grad=True)
y = layer1(x)
# 网络层2
z = layer2(y.detach())
o = z.sum()
o.backward()
'''
#第二种方法
# 网络层1
x = torch.randn(1,3,32,32, requires_grad=True)
y = layer1(x)
y.detach_()
# 网络层2
z = layer2(y)
o = z.sum()
o.backward()
'''

通过detach，将网络层进行了隔断，在backward的时候，传播到y就截止了，这样只能更新网络层2的参数。

In [47]:
layer1.weight.grad

In [48]:
layer2.weight.grad

tensor([[[[-104.8324, -104.7586, -103.7192],
          [-105.6326, -106.6240, -106.4484],
          [-103.5750, -105.7349, -105.7962]]]])

## evaluation

模型训练好之后进行评估(也称推理inference)有几点需要注意：
- 模型需要进入eval()模式，即调用model.eval()

Sets the module in evaluation mode.This has any effect only on certain modules. See documentations of particular modules for details of their behaviors in training/evaluation mode, if they are affected, e.g. Dropout, BatchNorm, etc.

- 模型不再需要进行grad，即调用torch.no_grad()
由于在求导过程中会消耗内存，故在推理过程中可以避免求导操作，节省内存。
```
with torch.no_grad():
   outputs = model(imgs)
```



In [1]:
from torchvision import datasets